In [ ]:
# =====================================================
# LSTM for 5-day accumulated volume using lagged volume sequences and VIX features
# =====================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers


In [ ]:

# -----------------------------
# 0) CONFIG
# -----------------------------
WINDOW = 30          # past trading days fed to the LSTM
BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-3
PATIENCE = 10        # early stopping
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)


In [ ]:

# -----------------------------
# 1) LOAD & MERGE
# -----------------------------
train = pd.read_csv("5_21_train.csv", parse_dates=["date"])
test  = pd.read_csv("5_21_test.csv", parse_dates=["date"])

train = train.sort_values("date").reset_index(drop=True)
test  = test.sort_values("date").reset_index(drop=True)
split_date = test["date"].min()

# Combine to ensure sequences that straddle the split are handled correctly
df = pd.concat([train, test], ignore_index=True).sort_values("date").reset_index(drop=True)


In [ ]:

# -----------------------------
# 2) Features
# -----------------------------
# We’ll only use log(sh_volume) as the single feature for the LSTM.
df["log_vol"] = np.log(df["sh_volume"])

# --- VIX features ---
df["vix_lag1"] = df["vix_close"].shift(1)
df["vix_change"] = df["vix_close"] - df["vix_lag1"]
df["vix_5d_ma"] = df["vix_close"].rolling(window=5).mean()

# --- Target ---
df["y_log"] = np.log(df["target_5d"])

# Drop rows with NaNs (start of series)
df = df.dropna(subset=["log_vol", "vix_close", "vix_lag1", "vix_change", "vix_5d_ma", "y_log"]).reset_index(drop=True)



In [ ]:

# -----------------------------
# 3) TRAIN / TEST SPLIT BY DATE (no leakage)
# -----------------------------
train_df = df[df["date"] < split_date].copy()
test_df  = df[df["date"] >= split_date].copy()


In [ ]:

# -----------------------------
# 4) SCALE FEATURES USING TRAIN ONLY
# -----------------------------
feature_cols = ["log_vol", "vix_close", "vix_lag1", "vix_change", "vix_5d_ma"]

scaler = StandardScaler()
scaler.fit(train_df[feature_cols])

train_df[feature_cols] = scaler.transform(train_df[feature_cols])
test_df[feature_cols]  = scaler.transform(test_df[feature_cols])


In [ ]:

# -----------------------------
# 5) BUILD SEQUENCES (WINDOW past days -> predict y at day t)
#    For each index t, X contains [log_vol_scaled at t-WINDOW+1 ... t]
#    and y contains y_log at t (target_5d is sum of t+1..t+5 -> safe)
# -----------------------------
def make_sequences(frame, window, feature_cols, target_col="y_log"):
    X_list, y_list, idx_list = [], [], []
    values = frame[feature_cols].values.astype(np.float32)
    targets = frame[target_col].values.astype(np.float32)
    for t in range(window - 1, len(frame)):
        X_list.append(values[t - window + 1 : t + 1])  # (window, features)
        y_list.append(targets[t])
        idx_list.append(frame["date"].iloc[t])
    X = np.array(X_list)
    y = np.array(y_list)
    idx = pd.to_datetime(idx_list)
    return X, y, idx

X_train, y_train, idx_train = make_sequences(train_df, WINDOW, feature_cols)
X_test,  y_test,  idx_test  = make_sequences(test_df,  WINDOW, feature_cols)

val_size = max(int(0.1 * len(X_train)), 1)
X_tr, y_tr = X_train[:-val_size], y_train[:-val_size]
X_val, y_val = X_train[-val_size:], y_train[-val_size:]

print(f"Train: {len(X_tr):,}  |  Val: {len(X_val):,}  |  Test: {len(X_test):,}")
print("Input shape:", X_tr.shape, " Target shape:", y_tr.shape)


In [ ]:

# -----------------------------
# 6) DEFINE LSTM MODEL
# -----------------------------
def build_model(window, n_features):
    model = models.Sequential([
        layers.Input(shape=(window, n_features)),
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(32),
        layers.Dropout(0.2),
        layers.Dense(1)
    ])
    model.compile(
        optimizer=optimizers.Adam(learning_rate=LR),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )
    return model

model = build_model(WINDOW, len(feature_cols))
model.summary()


In [ ]:

# -----------------------------
# 7) TRAIN
# -----------------------------
es = callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True)
lr_plateau = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    callbacks=[es, lr_plateau]
)


In [ ]:

# -----------------------------
# 8) EVALUATION
#     a) Log scale (y_log)
#     b) Original scale (exp)
# -----------------------------
# Predictions
y_pred_log = model.predict(X_test).ravel()
y_pred = np.exp(y_pred_log)
y_true = np.exp(y_test)

mse_log = mean_squared_error(y_test, y_pred_log)
rmse_log = np.sqrt(mse_log)
mae_log = mean_absolute_error(y_test, y_pred_log)
r2_log = r2_score(y_test, y_pred_log)

print("\nLog scale metrics:")
print(f"  RMSE (log): {rmse_log:.3f}")
print(f"  MAE  (log): {mae_log:.3f}")
print(f"  R²   (log): {r2_log:.3f}")

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_true, y_pred)
mape = (np.abs(y_pred - y_true) / y_true).mean() * 100

print("\nOriginal scale metrics:")
print(f"  RMSE: {rmse:,.0f}")
print(f"  MAE : {mae:,.0f}")
print(f"  MAPE: {mape:.2f}%")


In [ ]:

# -----------------------------
# 9) PLOTS
# -----------------------------
# Training history (loss)
plt.figure(figsize=(6,4))
plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Val")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE, log scale)")
plt.title("LSTM Training History (with VIX)")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.scatter(y_test, y_pred_log, alpha=0.6)
lo, hi = min(y_test.min(), y_pred_log.min()), max(y_test.max(), y_pred_log.max())
plt.plot([lo, hi], [lo, hi], 'r--')
plt.xlabel("Actual log(target_5d)")
plt.ylabel("Predicted log(target_5d)")
plt.title("LSTM: Actual vs Predicted (log scale, with VIX)")
plt.tight_layout()
plt.show()

ts_plot = pd.DataFrame({
    "date": idx_test,
    "actual_5d_volume": y_true,
    "predicted_5d_volume": y_pred
}).sort_values("date")

plt.figure(figsize=(12,5))
plt.plot(ts_plot["date"], ts_plot["actual_5d_volume"], label="Actual", alpha=0.85)
plt.plot(ts_plot["date"], ts_plot["predicted_5d_volume"], label="Predicted", alpha=0.85)
plt.xlabel("Date")
plt.ylabel("5-day accumulated volume")
plt.title("LSTM: Actual vs Predicted (original scale, with VIX)")
plt.legend()
plt.tight_layout()
plt.show()
